In [1]:
import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [2]:
class AWSDataset(Dataset):
    def __init__(self, data, input_days, output_days):
        self.data = data
        self.input_days = input_days
        self.output_days = output_days

    def __len__(self):
        return len(self.data) - self.input_days - self.output_days + 1

    def __getitem__(self, idx):
        input_data = self.data[idx : idx + self.input_days]
        output_data = self.data[idx + self.input_days : idx + self.input_days + self.output_days]
        return input_data, output_data

In [3]:
def preprocess_data(train_data, input_days, output_days):
    # 데이터 정규화
    scaler = MinMaxScaler()
    train_data = scaler.fit_transform(train_data)

    # 데이터셋 및 데이터 로더 생성
    dataset = AWSDataset(train_data, input_days, output_days)
    data_loader = DataLoader(dataset, batch_size=256, shuffle=True)
    return data_loader, scaler

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = torch.sigmoid(out)
        return out

In [5]:
def train_model(model, criterion, optimizer, data_loader, epochs):
    for epoch in range(epochs):
        model.train()
        for inputs, targets in data_loader:
            inputs, targets = inputs.float().to(device), targets.float().to(device)
            targets = targets.squeeze(-1)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = criterion(predictions, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
input_days = 48
output_days = 72
input_size = 1  # AWS 데이터의 feature 수
hidden_size = 64
num_layers = 2
learning_rate = 0.001
epochs = 100

In [8]:
train_data = pd.read_csv("mv_train_datafile.csv")  # 과거 4년간의 AWS 데이터 파일
train_data_temperatures = train_data[["기온(°C)"]].values  #

In [9]:
train_loader_temperatures, scaler_temperatures = preprocess_data(train_data_temperatures, input_days, output_days)

In [10]:
temperatures_model = LSTMModel(input_size, hidden_size, output_days, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(temperatures_model.parameters(), lr=learning_rate)

In [11]:
train_model(temperatures_model, criterion, optimizer, train_loader_temperatures, epochs)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100


In [12]:
train_data = pd.read_csv("mv_train_datafile.csv")  # 과거 4년간의 AWS 데이터 파일
train_data_wind_direction = train_data[["풍향(deg)"]].values  #

In [13]:
train_loader_wind_direction, scaler_wind_direction = preprocess_data(train_data_wind_direction, input_days, output_days)

In [14]:
wind_direction_model = LSTMModel(input_size, hidden_size, output_days, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(wind_direction_model.parameters(), lr=learning_rate)

In [15]:
train_model(wind_direction_model, criterion, optimizer, train_loader_wind_direction, epochs)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100


In [16]:
train_data = pd.read_csv("mv_train_datafile.csv")  # 과거 4년간의 AWS 데이터 파일
train_data_wind_speed = train_data[["풍속(m/s)"]].values  #

In [17]:
train_loader_wind_speed, scaler_wind_speed = preprocess_data(train_data_wind_speed, input_days, output_days)

In [18]:
wind_speed_model = LSTMModel(input_size, hidden_size, output_days, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(wind_speed_model.parameters(), lr=learning_rate)

In [19]:
train_model(wind_speed_model, criterion, optimizer, train_loader_wind_speed, epochs)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100


In [20]:
train_data = pd.read_csv("mv_train_datafile.csv")  # 과거 4년간의 AWS 데이터 파일
train_data_humidity = train_data[["습도(%)"]].values  #

In [21]:
train_loader_humidity, scaler_humidity = preprocess_data(train_data_humidity, input_days, output_days)

In [22]:
humidity_model = LSTMModel(input_size, hidden_size, output_days, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(humidity_model.parameters(), lr=learning_rate)

In [23]:
train_model(humidity_model, criterion, optimizer, train_loader_humidity, epochs)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100


In [24]:
filename = 'test_datafile.csv'
data = pd.read_csv(filename)

In [25]:
def predict_future(model, input_data, scaler):
    model.eval()
    with torch.no_grad():
        input_data = torch.tensor(input_data, dtype=torch.float32).unsqueeze(0).to(device)
        predictions = model(input_data).squeeze(0).cpu().numpy()
    return predictions

In [26]:
for k in range(17):
    for i in range(64):
        test_input_data_temperatures = data.iloc[i*120:i*120+48, 4]
        test_input_data_reshaped_temperatures = test_input_data_temperatures.to_numpy().reshape(-1, 1)
        predictions = predict_future(temperatures_model, test_input_data_reshaped_temperatures, scaler_temperatures)
        predictions = predictions[:72]
        data.iloc[7728*k+i*120+48:7728*k+i*120+120,4] = predictions.ravel()
    
        test_input_data_wind_direction = data.iloc[i*120:i*120+48, 5]
        test_input_data_reshaped_wind_direction = test_input_data_wind_direction.to_numpy().reshape(-1, 1)
        predictions = predict_future(wind_direction_model, test_input_data_reshaped_wind_direction, scaler_wind_direction)
        predictions = predictions[:72]
        data.iloc[7728*k+i*120+48:7728*k+i*120+120,5] = predictions.ravel()
    
        test_input_data_wind_speed = data.iloc[i*120:i*120+48, 6]
        test_input_data_reshaped_wind_speed = test_input_data_wind_speed.to_numpy().reshape(-1, 1)
        predictions = predict_future(wind_speed_model, test_input_data_reshaped_wind_speed, scaler_wind_speed)
        predictions = predictions[:72]
        data.iloc[7728*k+i*120+48:7728*k+i*120+120,6] = predictions.ravel()
    
        test_input_data_humidity = data.iloc[i*120:i*120+48, 7]
        test_input_data_reshaped_humidity = test_input_data_humidity.to_numpy().reshape(-1, 1)
        predictions = predict_future(humidity_model, test_input_data_reshaped_humidity, scaler_humidity)
        predictions = predictions[:72]
        data.iloc[7728*k+i*120+48:7728*k+i*120+120,7] = predictions.ravel()       

In [27]:
data.to_csv('real_all_awsdata(100).csv', index=False)